In [1]:
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## первая ф-я: уменьшение памяти под датафреймы.
## вторая        : считывание датафреймов
Запускаем только один раз(работает не быстро)

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
def read_data(INPUT_DIR = 'data/'):
    
    print("Reading files...")

    calendar = pd.read_csv(f"{INPUT_DIR}/calendar.csv").pipe(reduce_mem_usage)
    sell_prices = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv").pipe(reduce_mem_usage)

    sales_train_val = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv",).pipe(
        reduce_mem_usage
    )
    submission = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv").pipe(
        reduce_mem_usage
    )

    print("calendar shape:", calendar.shape)
    print("sell_prices shape:", sell_prices.shape)
    print("sales_train_val shape:", sales_train_val.shape)
    print("submission shape:", submission.shape)

    # calendar shape: (1969, 14)
    # sell_prices shape: (6841121, 4)
    # sales_train_val shape: (30490, 1919)
    # submission shape: (60980, 29)

    return calendar, sell_prices, sales_train_val, submission

In [4]:
calendar, sell_prices, sales_train_val, submission = read_data()

NUM_ITEMS = sales_train_val.shape[0]  # 30490
DAYS_PRED = submission.shape[1] - 1  # 28

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)
calendar shape: (1969, 14)
sell_prices shape: (6841121, 4)
sales_train_val shape: (30490, 1919)
submission shape: (60980, 29)


## File 1: “calendar.csv”  
Contains information about the dates the products are sold. 

date: The date in a “y-m-d” format. 

wm_yr_wk: The id of the week the date belongs to. 

weekday: The type of the day (Saturday, Sunday, …, Friday). 

wday: The id of the weekday, starting from Saturday. 

month: The month of the date. 

year: The year of the date. 

event_name_1: If the date includes an event, the name of this event. 

event_type_1: If the date includes an event, the type of this event. 

event_name_2: If the date includes a second event, the name of this event. 

event_type_2: If the date includes a second event, the type of this event. 

snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP3 purchases on the examined date. 1 indicates that SNAP purchases are allowed. SNAP - программа помощи нуждающимся в питании

In [7]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


## File 2: “sell_prices.csv” 

Contains information about the price of the products sold per store and date. 

store_id: The id of the store where the product is sold.  

item_id: The id of the product. 

wm_yr_wk: The id of the week. 

sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).   

In [8]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


## File 3: “sales_train.csv”  

Contains the historical daily unit sales data per product and store. 

item_id: The id of the product. 

dept_id: The id of the department the product belongs to. 

cat_id: The id of the category the product belongs to. 

store_id: The id of the store where the product is sold. 

state_id: The State where the store is located. 

d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.  

In [9]:
sales_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


<b>Файл для отправки. Задание хакатона - спрогнозировать количество продаж с F1 по F28 дни(даны продажи за предыд. несколько лет).<br>
этот файл поделен на 2 части:<br>
первая(id заканчиваются на validation) - для них F1 - F28 это d_1914 - d_1941(продолжение traina);<br>
вторая(id заканчиваются на evaluation) F1-F28 - это d_1942 - d_1969<br>
всего 30 490 * 2 строк.<br>
Сейчас мы прогнозируем для validation и отправляем на проверку(до конца мая).<br>
Потом будут предоставлены реальные значения для validation и надо будет предсказать evaluation, но проверить скор на kaggle уже будет нельзя

In [10]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Baseline. 
Идея банальна: берем предыдущий месяц/90дней/год и для каждого товара усредняем по дням недели количество продаж.<br>
joinim<br>
from https://www.kaggle.com/chrisrichardmiles/numpy-pandas-challenge-current-leader-71444

In [11]:
# We need to select only the last 28 days
last_28 = sales_train_val.iloc[:, pd.np.r_[0,5,-28:0]] # we include 0, and 5 to get the id and state id columns
print(last_28.shape)
last_28.head()

(30490, 30)


,id,state_id,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,CA,1,0,0,0,0,0,1,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,CA,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,CA,0,0,0,0,0,0,1,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,CA,0,0,0,0,3,1,2,1,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,CA,1,0,4,4,0,1,4,0,...,2,1,1,0,1,1,2,2,2,4


In [12]:
last_90 = sales_train_val.iloc[:, pd.np.r_[0,5,-90:0]] # we include 0, and 5 to get the id and state id columns
last_365 = sales_train_val.iloc[:, pd.np.r_[0,5,-365:0]] # we include 0, and 5 to get the id and state id columns

In [13]:
def add_snap_col(df_in):
    """adds a 'snap_day' column to a dataframe that contains a state_id column and the columns 'snap_CA', 'snap_TX', 
    and snap_WI"""
    df = pd.get_dummies(df_in, columns=['state_id'])
    df['snap_day'] = (df.snap_CA * df.state_id_CA) + (df.snap_WI * df.state_id_WI) + (df.snap_TX * df.state_id_TX)
    del df['state_id_WI'], df['state_id_CA'], df['state_id_TX']
    return df



def melt_merge_snap(df, calendar):
    df = df.melt(['id', 'state_id'], var_name='d', value_name='demand')
    df = df.merge(calendar)
    df = add_snap_col(df)
    return df

In [14]:
last_28 = melt_merge_snap(last_28, calendar)
last_90 = melt_merge_snap(last_90, calendar)
last_365 = melt_merge_snap(last_365, calendar)

In [15]:
last_28.head()

,id,d,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap_day
0,HOBBIES_1_001_CA_1_validation,d_1886,1,2016-03-28,11609,Monday,3,3,2016,NaN,NaN,NaN,NaN,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,d_1886,1,2016-03-28,11609,Monday,3,3,2016,NaN,NaN,NaN,NaN,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,d_1886,0,2016-03-28,11609,Monday,3,3,2016,NaN,NaN,NaN,NaN,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,d_1886,0,2016-03-28,11609,Monday,3,3,2016,NaN,NaN,NaN,NaN,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,d_1886,1,2016-03-28,11609,Monday,3,3,2016,NaN,NaN,NaN,NaN,0,0,0,0


In [16]:
by_weekday = last_28.groupby(['id','wday'])['demand'].mean()

In [17]:
by_weekday

id                               wday
FOODS_1_001_CA_1_validation      1       0.75
                                 2       0.50
                                 3       3.00
                                 4       0.75
                                 5       1.50
                                         ... 
HOUSEHOLD_2_516_WI_3_validation  3       0.00
                                 4       0.00
                                 5       0.00
                                 6       0.00
                                 7       0.00
Name: demand, Length: 213430, dtype: float64

In [18]:
# со snapом
by_weekday_snap = last_28.groupby(['id', 'wday', 'snap_day'])['demand'].mean()
by_weekday_snap_90 = last_90.groupby(['id', 'wday', 'snap_day'])['demand'].mean()
by_weekday_snap_365 = last_365.groupby(['id', 'wday', 'snap_day'])['demand'].mean()

In [19]:
by_weekday_snap

id                               wday  snap_day
FOODS_1_001_CA_1_validation      1     0           1.500000
                                       1           0.000000
                                 2     0           0.000000
                                       1           1.000000
                                 3     0           2.666667
                                                     ...   
HOUSEHOLD_2_516_WI_3_validation  5     1           0.000000
                                 6     0           0.000000
                                       1           0.000000
                                 7     0           0.000000
                                       1           0.000000
Name: demand, Length: 426860, dtype: float64

In [20]:
def get_sub(submission, calendar):
    """returns a tidy dataframe version of the sample submission, merged with the calendar data, 
    without the 'demand' column. It can be used to join to a group by series to make predictions  """
    # make a copy of the sample submission
    sub = submission.copy()
    # change the column names to match the last 28 days
    sub.columns = ['id'] + ['d_' + str(1914+x) for x in range(28)]
    # select only the rows with an id with the validation tag
    sub = sub.loc[sub.id.str.contains('validation')]
    # melt this dataframe and merge it with the calendar so we can join it with group_by series we create
    sub = sub.melt('id', var_name='d', value_name='demand')
    sub = sub.merge(calendar)
    
    
    # add state_id column so that we can add the snap_day column
    sub['state_id'] = sub.id.str.split('_', expand=True)[3]
    
    # add the snap_day column
    sub = add_snap_col(sub)
    
    return sub.drop('demand', axis='columns')



def join_sub_groupby(sub, group):
    """ 
    Joins the sub dataframe created by get_sub to a groupby series
    """
    return sub.join(group, on=group.index.names)



def make_sub(df_in, submission, filename='submission.csv'): 
    """
    Takes a dataframe in the form given by join_sub_groupby, or any dataframe with the proper index and and 'd' colums.
    returns a csv submission file in the correct format
    """
    # pivot df to get it into the proper format for submission
    df = df_in.pivot(index='id', columns='d', values='demand')
    # need to reset index to take care of columns. comment next line out to see what i mean 
    df.reset_index(inplace=True)
    
    sub = submission[['id']].copy()    
    sub = sub.merge(df)    
    # we must copy the dataframe to match the format of the submission file which is twice as long as what we have
    sub = pd.concat([sub, sub], axis=0)
    # reset the id colum to have the same values as the sample submission
    sub['id'] = submission.id.values
    # rename the columns to match the sample submission format 
    sub.columns = ['id'] + ['F' + str(i) for i in range(1,29)]
    
    sub.to_csv(filename, index=False)  

In [21]:
sub = get_sub(submission, calendar)
sub.head()

,id,d,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap_day
0,HOBBIES_1_001_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0


In [22]:
df_final_model_1 = join_sub_groupby(sub, by_weekday)
df_final_model_2 = join_sub_groupby(sub, by_weekday_snap)
df_final_model_3 = join_sub_groupby(sub, by_weekday_snap_90)
df_final_model_4 = join_sub_groupby(sub, by_weekday_snap_365)

In [23]:
df_final_model_1.head()

,id,d,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap_day,demand
0,HOBBIES_1_001_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0,0.50
1,HOBBIES_1_002_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0,0.25
2,HOBBIES_1_003_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0,0.25
3,HOBBIES_1_004_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0,1.75
4,HOBBIES_1_005_CA_1_validation,d_1914,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,0,0.75


In [23]:
make_sub(df_final_model_1, submission, filename='model1sub.csv')
make_sub(df_final_model_2, submission, filename='model2sub.csv')
make_sub(df_final_model_3, submission, filename='model3sub.csv')
make_sub(df_final_model_4, submission, filename='model4sub.csv')

copy-past from https://www.kaggle.com/dhananjay3/wrmsse-evaluator-with-extra-features<br>
Класс для посчета метрики(с кодом можно не разбираться:) )

In [25]:
from typing import Union

import numpy as np
import pandas as pd
from tqdm.auto import tqdm as tqdm

class WRMSSEEvaluator(object):
    
    group_ids = ( 'all_id', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id',
        ['state_id', 'cat_id'],  ['state_id', 'dept_id'], ['store_id', 'cat_id'],
        ['store_id', 'dept_id'], ['item_id', 'state_id'], ['item_id', 'store_id'])

    def __init__(self, 
                 train_df: pd.DataFrame, 
                 valid_df: pd.DataFrame, 
                 calendar: pd.DataFrame, 
                 prices: pd.DataFrame):
        '''
        intialize and calculate weights
        '''
        self.calendar = calendar
        self.prices = prices
        self.train_df = train_df
        self.valid_df = valid_df
        self.train_target_columns = [i for i in self.train_df.columns if i.startswith('d_')]
        self.weight_columns = self.train_df.iloc[:, -28:].columns.tolist()

        self.train_df['all_id'] = "all"

        self.id_columns = [i for i in self.train_df.columns if not i.startswith('d_')]
        self.valid_target_columns = [i for i in self.valid_df.columns if i.startswith('d_')]

        if not all([c in self.valid_df.columns for c in self.id_columns]):
            self.valid_df = pd.concat([self.train_df[self.id_columns], self.valid_df],
                                      axis=1, 
                                      sort=False)
        self.train_series = self.trans_30490_to_42840(self.train_df, 
                                                      self.train_target_columns, 
                                                      self.group_ids)
        self.valid_series = self.trans_30490_to_42840(self.valid_df, 
                                                      self.valid_target_columns, 
                                                      self.group_ids)
        self.weights = self.get_weight_df()
        self.scale = ((self.train_series.iloc[:, 1:].values - self.train_series.iloc[:, :-1].values) ** 2).mean(axis=1)
        self.train_series = None
        self.train_df = None
        self.prices = None
        self.calendar = None

    def get_name(self, i):
        '''
        convert a str or list of strings to unique string 
        used for naming each of 42840 series
        '''
        if type(i) == str or type(i) == int:
            return str(i)
        else:
            return "--".join(i)
    
    def get_weight_df(self) -> pd.DataFrame:
        """
        returns weights for each of 42840 series in a dataFrame
        """
        day_to_week = self.calendar.set_index("d")["wm_yr_wk"].to_dict()
        weight_df = self.train_df[["item_id", "store_id"] + self.weight_columns].set_index(
            ["item_id", "store_id"]
        )
        weight_df = (
            weight_df.stack().reset_index().rename(columns={"level_2": "d", 0: "value"})
        )
        weight_df["wm_yr_wk"] = weight_df["d"].map(day_to_week)
        weight_df = weight_df.merge(
            self.prices, how="left", on=["item_id", "store_id", "wm_yr_wk"]
        )
        weight_df["value"] = weight_df["value"] * weight_df["sell_price"]
        weight_df = weight_df.set_index(["item_id", "store_id", "d"]).unstack(level=2)[
            "value"
        ]
        weight_df = weight_df.loc[
            zip(self.train_df.item_id, self.train_df.store_id), :
        ].reset_index(drop=True)
        weight_df = pd.concat(
            [self.train_df[self.id_columns], weight_df], axis=1, sort=False
        )
        weights_map = {}
        for i, group_id in enumerate(tqdm(self.group_ids, leave=False)):
            lv_weight = weight_df.groupby(group_id)[self.weight_columns].sum().sum(axis=1)
            lv_weight = lv_weight / lv_weight.sum()
            for i in range(len(lv_weight)):
                weights_map[self.get_name(lv_weight.index[i])] = np.array(
                    [lv_weight.iloc[i]]
                )
        weights = pd.DataFrame(weights_map).T / len(self.group_ids)

        return weights

    def trans_30490_to_42840(self, df, cols, group_ids, dis=False):
        '''
        transform 30490 sries to all 42840 series
        '''
        series_map = {}
        for i, group_id in enumerate(tqdm(self.group_ids, leave=False, disable=dis)):
            tr = df.groupby(group_id)[cols].sum()
            for i in range(len(tr)):
                series_map[self.get_name(tr.index[i])] = tr.iloc[i].values
        return pd.DataFrame(series_map).T
    
    def get_rmsse(self, valid_preds) -> pd.Series:
        '''
        returns rmsse scores for all 42840 series
        '''
        score = ((self.valid_series - valid_preds) ** 2).mean(axis=1)
        rmsse = (score / self.scale).map(np.sqrt)
        return rmsse

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds],
                                axis=1, 
                                sort=False)
        valid_preds = self.trans_30490_to_42840(valid_preds, 
                                                self.valid_target_columns, 
                                                self.group_ids, 
                                                True)
        self.rmsse = self.get_rmsse(valid_preds)
        self.contributors = pd.concat([self.weights, self.rmsse], 
                                      axis=1, 
                                      sort=False).prod(axis=1)
        return np.sum(self.contributors)

In [26]:
train_fold_df = sales_train_val.iloc[:, :-28]
valid_fold_df = sales_train_val.iloc[:, -28:].copy()

e = WRMSSEEvaluator(train_fold_df, valid_fold_df, calendar, sell_prices)
del train_fold_df, sales_train_val, calendar, sell_prices

In [27]:
valid_preds = np.random.randint(4, size=valid_fold_df.shape)
e.score(valid_preds)

2.586834049818075